# SVM

In [ ]:
import pandas as pd
import numpy as np

freq = 'Daily'
stock = 'AAPL'
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.values
predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

## Linear Kernel

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=False)
sm = SMOTE(sampling_strategy='minority',random_state=42,k_neighbors=5)
X_res,y_res = sm.fit_resample(X_train,y_train)
pipe = make_pipeline(StandardScaler(),SVC(kernel='linear'))
pipe.fit(X_res,y_res)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.39      0.47       230
           1       0.60      0.77      0.68       278

    accuracy                           0.60       508
   macro avg       0.59      0.58      0.57       508
weighted avg       0.59      0.60      0.58       508



## Polynomial Kernel

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=False)
pipe = make_pipeline(StandardScaler(),SVC(kernel='poly',degree=4))
pipe.fit(X_res,y_res)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.52      0.53      0.52       230
           1       0.61      0.60      0.61       278

    accuracy                           0.57       508
   macro avg       0.57      0.57      0.57       508
weighted avg       0.57      0.57      0.57       508



In [8]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.56      0.43      0.49       230
           1       0.61      0.72      0.66       278

    accuracy                           0.59       508
   macro avg       0.59      0.58      0.58       508
weighted avg       0.59      0.59      0.58       508



## RBF Kernel

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=False)
pipe = make_pipeline(StandardScaler(),SVC(kernel='rbf'))
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.52      0.23      0.32       230
           1       0.57      0.83      0.67       278

    accuracy                           0.56       508
   macro avg       0.54      0.53      0.50       508
weighted avg       0.55      0.56      0.51       508



## Sigmoid Kernel

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=False)
pipe = make_pipeline(StandardScaler(),SVC(kernel= 'sigmoid'))
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.39      0.25      0.31       230
           1       0.52      0.67      0.59       278

    accuracy                           0.48       508
   macro avg       0.46      0.46      0.45       508
weighted avg       0.46      0.48      0.46       508

